# mario with reinforcement learning

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import GrayScaleObservation
import numpy as np
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

ImportError: cannot import name 'GrayScaleObservation' from 'gymnasium.wrappers' (/opt/homebrew/Caskroom/miniforge/base/envs/super-mario-rl/lib/python3.10/site-packages/gymnasium/wrappers/__init__.py)

: 

In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback


In [3]:
import os
from matplotlib import pyplot as plt

In [4]:
class TrainAndLoggingCallback(BaseCallback):
    """Callback for saving model checkpoints during training"""
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'best_model_{self.n_calls}')
            self.model.save(model_path)
            if self.verbose > 0:
                print(f"Saved model at step {self.n_calls}")
        return True


In [5]:
CHECKPOINT_DIR = './data/mario/train/'
LOG_DIR = './data/mario/logs/'


In [6]:
# Create the callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)


In [7]:
# Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', render_mode='human', apply_api_compatibility=True)
# Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

/opt/homebrew/Caskroom/miniforge/base/envs/super-mario-rl/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/super-mario-rl/lib/python3.10/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [8]:
# Test with random actions
done = True
for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    done = done or truncated
    env.render()

env.close()


/opt/homebrew/Caskroom/miniforge/base/envs/super-mario-rl/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/opt/homebrew/Caskroom/miniforge/base/envs/super-mario-rl/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(
AGX: exceeded compiled variants footprint limit


AttributeError: 'CocoaAlternateEventLoop' object has no attribute 'platform_event_loop'

In [9]:
# 1. Base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0', render_mode='human', apply_api_compatibility=True)

# 2. Simplified controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

# 3. Grayscale frames
env = GrayScaleObservation(env, keep_dim=True)

# 4. Wrap for vectorization
env = DummyVecEnv([lambda: env])

# 5. Stack frames
env = VecFrameStack(env, 4, channels_order='last')


AssertionError: Expected env to be a `gymnasium.Env` but got <class 'nes_py.wrappers.joypad_space.JoypadSpace'>